# TransE的pytorch实现

TransE不是神经网络模型，它无法理解句子的含义，因而得分不高，随便跑跑mrr大概在42左右，调下参大概有45-50。

TransE的优势在于不吃算力，仅靠CPU就能运行，这里我写的GPU版本速度更快。而且代码简单。

又试了一下，我已经复现不出自己的得分了，太玄学了，大概策略就是先norm取1得到最优模型，加载该模型并norm改为2训练得到最优模型，再加载该模型norm取3得到最优模型，再反复调整后最优分数就是50左右；
在算力允许的情况下，batchsize尽可能大，比如10万-150万（就是全部）；然后就是embedding维度我感觉影响不大，100维包含的信息足够丰富了，试过256和512，貌似大一点有时要好一些，不太确定，太大了会比较费算力；学习率粗调可以在0.01-0.001之间，微调就在0.001-0.0001之间吧。

原作者github下载：https://github.com/renqi1/TransE_Pytorch_OpenBG500


In [1]:
import torch
from torch import nn
from torch.utils import data
import numpy as np
import tqdm

## 构建数据集

In [2]:
# 训练集和验证集
class TripleDataset(data.Dataset):
    def __init__(self, ent2id, rel2id, triple_data_list):
        self.ent2id = ent2id
        self.rel2id = rel2id
        self.data = triple_data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        head, relation, tail = self.data[index]
        head_id = self.ent2id[head]
        relation_id = self.rel2id[relation]
        tail_id = self.ent2id[tail]
        return head_id, relation_id, tail_id

# 测试集    
class TestDataset(data.Dataset):
    def __init__(self, ent2id, rel2id, test_data_list):
        self.ent2id = ent2id
        self.rel2id = rel2id
        self.data = test_data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        head, relation = self.data[index]
        head_id = self.ent2id[head]
        relation_id = self.rel2id[relation]
        return head_id, relation_id

## TransE模型

In [3]:
class TransE(nn.Module):

    def __init__(self, entity_num, relation_num, norm=1, dim=100):
        super(TransE, self).__init__()
        self.norm = norm
        self.dim = dim
        self.entity_num = entity_num
        self.entities_emb = self._init_emb(entity_num)
        self.relations_emb = self._init_emb(relation_num)

    def _init_emb(self, num_embeddings):
        embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=self.dim)
        uniform_range = 6 / np.sqrt(self.dim)
        embedding.weight.data.uniform_(-uniform_range, uniform_range)
        embedding.weight.data = torch.div(embedding.weight.data, embedding.weight.data.norm(p=2, dim=1, keepdim=True))
        return embedding

    def forward(self, positive_triplets: torch.LongTensor, negative_triplets: torch.LongTensor):
        positive_distances = self._distance(positive_triplets)
        negative_distances = self._distance(negative_triplets)
        return positive_distances, negative_distances

    def _distance(self, triplets):
        heads = self.entities_emb(triplets[:, 0])
        relations = self.relations_emb(triplets[:, 1])
        tails = self.entities_emb(triplets[:, 2])
        return (heads + relations - tails).norm(p=self.norm, dim=1)

    def link_predict(self, head, relation, tail=None, k=10):
        # h_add_r: [batch size, embed size] -> [batch size, 1, embed size] -> [batch size, entity num, embed size]
        h_add_r = self.entities_emb(head) + self.relations_emb(relation)
        h_add_r = torch.unsqueeze(h_add_r, dim=1)
        h_add_r = h_add_r.expand(h_add_r.shape[0], self.entity_num, self.dim)
        # embed_tail: [batch size, embed size] -> [batch size, entity num, embed size]
        embed_tail = self.entities_emb.weight.data.expand(h_add_r.shape[0], self.entity_num, self.dim)
        # values: [batch size, k] scores, the smaller, the better
        # indices: [batch size, k] indices of entities ranked by scores
        values, indices = torch.topk(torch.norm(h_add_r - embed_tail, dim=2), k=self.entity_num, dim=1, largest=False)
        if tail is not None:
            tail = tail.view(-1, 1)
            rank_num = torch.eq(indices, tail).nonzero().permute(1, 0)[1]+1
            rank_num[rank_num > 9] = 10000
            mrr = torch.sum(1/rank_num)
            hits_1_num = torch.sum(torch.eq(indices[:, :1], tail)).item()
            hits_3_num = torch.sum(torch.eq(indices[:, :3], tail)).item()
            hits_10_num = torch.sum(torch.eq(indices[:, :10], tail)).item()
            return mrr, hits_1_num, hits_3_num, hits_10_num     # 返回一个batchsize, mrr的和，hit@k的和
        return indices[:, :k]

    def evaluate(self, data_loader, dev_num=5000.0):
        mrr_sum = hits_1_nums = hits_3_nums = hits_10_nums = 0
        for heads, relations, tails in tqdm.tqdm(data_loader):
            mrr_sum_batch, hits_1_num, hits_3_num, hits_10_num = self.link_predict(heads.cuda(), relations.cuda(), tails.cuda())
            mrr_sum += mrr_sum_batch
            hits_1_nums += hits_1_num
            hits_3_nums += hits_3_num
            hits_10_nums += hits_10_num
        return mrr_sum/dev_num, hits_1_nums/dev_num, hits_3_nums/dev_num, hits_10_nums/dev_num

## 设置参数

In [4]:
# batchsize增大，得分略有上升
train_batch_size = 1500000
dev_batch_size = 20  # 显存不够就调小
test_batch_size = 20
epochs = 40
margin = 1
print_frequency = 5  # 每多少step输出一次信息
validation = True  # 是否验证，验证比较费时
dev_interval = 5  # 每多少轮验证一次，微调设小一点，会保存最佳权重
best_mrr = 0
learning_rate = 0.001  # 学习率建议粗调0.01-0.001，精调0.001-0.0001
distance_norm = 3  # 论文是L1距离效果不好，取2或3效果好
embedding_dim = 100  # 维度增大可能会有提升，我感觉没用，100维包含的信息足够丰富

## 加载数据集

In [5]:
with open('OpenBG500/OpenBG500_entity2text.tsv', 'r', encoding='utf-8') as fp:
    dat = fp.readlines()
    lines = [line.strip('\n').split('\t') for line in dat]
ent2id = {line[0]: i for i, line in enumerate(lines)}
id2ent = {i: line[0] for i, line in enumerate(lines)}
with open('OpenBG500/OpenBG500_relation2text.tsv', 'r', encoding='utf-8') as fp:
    dat = fp.readlines()
    lines = [line.strip().split('\t') for line in dat]
rel2id = {line[0]: i for i, line in enumerate(lines)}
with open('OpenBG500/OpenBG500_train.tsv', 'r', encoding='utf-8') as fp:
    dat = fp.readlines()
    train = [line.strip('\n').split('\t') for line in dat]
with open('OpenBG500/OpenBG500_dev.tsv', 'r', encoding='utf-8') as fp:
    dat = fp.readlines()
    dev = [line.strip('\n').split('\t') for line in dat]
with open('OpenBG500/OpenBG500_test.tsv', 'r', encoding='utf-8') as fp:
    test = fp.readlines()
    test = [line.strip('\n').split('\t') for line in test]
# 构建数据集
train_dataset = TripleDataset(ent2id, rel2id, train)
dev_dataset = TripleDataset(ent2id, rel2id, dev)
train_data_loader = data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
dev_data_loader = data.DataLoader(dev_dataset, batch_size=dev_batch_size)
test_dataset = TestDataset(ent2id, rel2id, test)
test_data_loader = data.DataLoader(test_dataset, batch_size=test_batch_size)

## 训练和验证

In [6]:
# 构建模型
model = TransE(len(ent2id), len(rel2id), norm=distance_norm, dim=embedding_dim).cuda()
# model.load_state_dict(torch.load('transE_best.pth'))
# 优化器adam
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 损失函数， 对于本例，loss=max(0, (pd-nd)+1)， 负样本距离越小，正样本距离越大越好
criterion = nn.MarginRankingLoss(margin=margin, reduction='mean')
# 开始训练
print('start training...')
for epoch in range(epochs):
    all_loss = 0
    for i, (local_heads, local_relations, local_tails) in enumerate(train_data_loader):

        positive_triples = torch.stack((local_heads, local_relations, local_tails), dim=1).cuda()

        # 生成负样本
        head_or_tail = torch.randint(high=2, size=local_heads.size())
        random_entities = torch.randint(high=len(ent2id), size=local_heads.size())
        broken_heads = torch.where(head_or_tail == 1, random_entities, local_heads)
        broken_tails = torch.where(head_or_tail == 0, random_entities, local_tails)
        negative_triples = torch.stack((broken_heads, local_relations, broken_tails), dim=1).cuda()

        # # 生成负样本, 只打乱tail
        # random_entities = torch.randint(high=len(ent2id), size=local_heads.size())
        # negative_triples = torch.stack((random_entities, local_relations, random_entities), dim=1).cuda()

        optimizer.zero_grad()
        pd, nd = model(positive_triples, negative_triples)
        # pd要尽可能小， nd要尽可能大
        loss = criterion(pd, nd, torch.tensor([-1], dtype=torch.long).cuda())
        loss.backward()
        all_loss += loss.data
        optimizer.step()
        if i % print_frequency == 0:
            print(
                f"epoch:{epoch}/{epochs}, step:{i}/{len(train_data_loader)}, loss={loss.item()}, avg_loss={all_loss / (i + 1)}")
    print(f"epoch:{epoch}/{epochs}, all_loss={all_loss}")

    # 验证
    if validation and (epoch + 1) % dev_interval == 0:
        print('testing...')
        improve = ''
        mrr, hits1, hits3, hits10 = model.evaluate(dev_data_loader)
        if mrr >= best_mrr:
            best_mrr = mrr
            improve = '*'
            torch.save(model.state_dict(), 'transE_best.pth')
        torch.save(model.state_dict(), 'transE_latest.pth')
        print(f'mrr: {mrr}, hit@1: {hits1}, hit@3: {hits3}, hit@10: {hits10}  {improve}')
    if not validation:
        torch.save(model.state_dict(), 'transE_latest.pth')

e:\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


start training...
epoch:0/40, step:0/1, loss=1.0005627870559692, avg_loss=1.0005627870559692
epoch:0/40, all_loss=1.0005627870559692
epoch:1/40, step:0/1, loss=0.9972615838050842, avg_loss=0.9972615838050842
epoch:1/40, all_loss=0.9972615838050842
epoch:2/40, step:0/1, loss=0.9940488934516907, avg_loss=0.9940488934516907
epoch:2/40, all_loss=0.9940488934516907
epoch:3/40, step:0/1, loss=0.9907407760620117, avg_loss=0.9907407760620117
epoch:3/40, all_loss=0.9907407760620117
epoch:4/40, step:0/1, loss=0.9875441193580627, avg_loss=0.9875441193580627
epoch:4/40, all_loss=0.9875441193580627
testing...


100%|██████████| 250/250 [00:07<00:00, 33.84it/s]


mrr: 0.00010000012844102457, hit@1: 0.0, hit@3: 0.0, hit@10: 0.0002  *
epoch:5/40, step:0/1, loss=0.9842342138290405, avg_loss=0.9842342138290405
epoch:5/40, all_loss=0.9842342138290405
epoch:6/40, step:0/1, loss=0.9809467196464539, avg_loss=0.9809467196464539
epoch:6/40, all_loss=0.9809467196464539
epoch:7/40, step:0/1, loss=0.977801501750946, avg_loss=0.977801501750946
epoch:7/40, all_loss=0.977801501750946
epoch:8/40, step:0/1, loss=0.9745674133300781, avg_loss=0.9745674133300781
epoch:8/40, all_loss=0.9745674133300781
epoch:9/40, step:0/1, loss=0.9713444113731384, avg_loss=0.9713444113731384
epoch:9/40, all_loss=0.9713444113731384
testing...


100%|██████████| 250/250 [00:07<00:00, 33.97it/s]


mrr: 0.0003054547414649278, hit@1: 0.0, hit@3: 0.0002, hit@10: 0.0012  *
epoch:10/40, step:0/1, loss=0.968007504940033, avg_loss=0.968007504940033
epoch:10/40, all_loss=0.968007504940033


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x00000220D5B68910>>
Traceback (most recent call last):
  File "e:\anaconda3\envs\NLP\lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


epoch:11/40, step:0/1, loss=0.9648275375366211, avg_loss=0.9648275375366211
epoch:11/40, all_loss=0.9648275375366211
epoch:12/40, step:0/1, loss=0.9616021513938904, avg_loss=0.9616021513938904
epoch:12/40, all_loss=0.9616021513938904
epoch:13/40, step:0/1, loss=0.9584448933601379, avg_loss=0.9584448933601379
epoch:13/40, all_loss=0.9584448933601379


## 预测

In [7]:
predict_all = []
# 构建模型
model = TransE(len(ent2id), len(rel2id), norm=distance_norm, dim=embedding_dim).cuda()
model.load_state_dict(torch.load('transE_best.pth'))
for heads, relations in tqdm.tqdm(test_data_loader):
    # 预测的id,结果为tensor(batch_size*10)
    predict_id = model.link_predict(heads.cuda(), relations.cuda())
    # 结果取到cpu并转为一行的list以便迭代
    predict_list = predict_id.cpu().numpy().reshape(1,-1).squeeze(0).tolist()
    # id转为实体
    predict_ent = map(lambda x: id2ent[x], predict_list)
    # 保存结果
    predict_all.extend(predict_ent)
print('prediction finished !')

100%|██████████| 250/250 [00:07<00:00, 33.75it/s]

prediction finished !


## 写入文件并保存

In [8]:
# 写入文件，按提交要求
with open('submission.tsv', 'w', encoding='utf-8') as f:
    for i in range(len(test)):
        # 直接writelines没有空格分隔，手工加分割符，得按提交格式来
        list = [x + '\t' for x in test[i]] + [x + '\n' if i == 9 else x + '\t' for i, x in enumerate(predict_all[i*10:i*10+10])]
        f.writelines(list)
print('file saved !')

file saved !
